## Assignment 1
### Name: Subhash Agrawal
### Roll Number: 23CS60R67

In [28]:
# import all the necessary libraries here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [29]:
df = pd.read_excel('../../dataset/logistic-regression/Pumpkin_Seeds_Dataset.xlsx')
print(df.shape) 
df

(2500, 13)


,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Convex_Area,Equiv_Diameter,Eccentricity,Solidity,Extent,Roundness,Aspect_Ration,Compactness,Class
0,56276,888.242,326.1485,220.2388,56831,267.6805,0.7376,0.9902,0.7453,0.8963,1.4809,0.8207,Çerçevelik
1,76631,1068.146,417.1932,234.2289,77280,312.3614,0.8275,0.9916,0.7151,0.8440,1.7811,0.7487,Çerçevelik
2,71623,1082.987,435.8328,211.0457,72663,301.9822,0.8749,0.9857,0.7400,0.7674,2.0651,0.6929,Çerçevelik
3,66458,992.051,381.5638,222.5322,67118,290.8899,0.8123,0.9902,0.7396,0.8486,1.7146,0.7624,Çerçevelik
4,66107,998.146,383.8883,220.4545,67117,290.1207,0.8187,0.9850,0.6752,0.8338,1.7413,0.7557,Çerçevelik
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,79637,1224.710,533.1513,190.4367,80381,318.4289,0.9340,0.9907,0.4888,0.6672,2.7996,0.5973,Ürgüp Sivrisi
2496,69647,1084.318,462.9416,191.8210,70216,297.7874,0.9101,0.9919,0.6002,0.7444,2.4134,0.6433,Ürgüp Sivrisi
2497,87994,1210.314,507.2200,222.1872,88702,334.7199,0.8990,0.9920,0.7643,0.7549,2.2828,0.6599,Ürgüp Sivrisi
2498,80011,1182.947,501.9065,204.7531,80902,319.1758,0.9130,0.9890,0.7374,0.7185,2.4513,0.6359,Ürgüp Sivrisi


# Preprocessing the Data

In [30]:
#Store the features & Targets in X & Y respectively and Update the Output to class 0 and 1
X = df.drop("Class", axis=1)
y = df["Class"].map({'Çerçevelik':0,'Ürgüp Sivrisi':1})

In [31]:
print(X,y)

       Area  Perimeter  Major_Axis_Length  Minor_Axis_Length  Convex_Area  \
0     56276    888.242           326.1485           220.2388        56831   
1     76631   1068.146           417.1932           234.2289        77280   
2     71623   1082.987           435.8328           211.0457        72663   
3     66458    992.051           381.5638           222.5322        67118   
4     66107    998.146           383.8883           220.4545        67117   
...     ...        ...                ...                ...          ...   
2495  79637   1224.710           533.1513           190.4367        80381   
2496  69647   1084.318           462.9416           191.8210        70216   
2497  87994   1210.314           507.2200           222.1872        88702   
2498  80011   1182.947           501.9065           204.7531        80902   
2499  84934   1159.933           462.8951           234.5597        85781   

      Equiv_Diameter  Eccentricity  Solidity  Extent  Roundness  \
0       

In [32]:
#split data intp training, testing and validation srt
X_train, X_testval, y_train, y_testval = train_test_split(X, y, test_size=0.5, random_state=0)    #50% train
X_test, X_val, y_test, y_val = train_test_split(X_testval, y_testval, test_size=0.6, random_state=0) #20% test and 30% validation

In [33]:
#Standardize Data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_standard = scaler.fit_transform(X_train)
x_test_standard = scaler.transform(X_test)
x_val_standard = scaler.transform(X_val)

In [34]:
y_train=y_train[:,np.newaxis]
y_train.shape

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15192\60764582.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y_train=y_train[:,np.newaxis]


(1250, 1)

# Supporting Functions for Training the Model

In [35]:
# Define the sigmoid function
def sigmoid(z):
    p=np.exp(-z)
    return 1 / (1 + p)

In [36]:
# Compute Log Loss

def logloss(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  log_loss = -1 * np.mean( y_true*np.log10(y_pred) + (1-y_true)*np.log10(1-y_pred)) 
  return log_loss

In [37]:
#function to compute the gardient w.r.to w
def gradient_dw(x, y, w, b, alpha, N):
  dw = x * (y-sigmoid(np.dot(w.T,x)+b)) - ((alpha*w*w)/N)
  return dw

# Training the Model based on Train Data

In [38]:
#TRAIN THE MODEL
def train(X_train, y_train, epochs, alpha):
  w = np.zeros_like(X_train[0])    #coefficient
  b = 0       #bias
  N = len(X_train)
  log_loss_train = []

  for i in range(0, epochs):

    for j in range(N):
#       grad_dw = X_train[j] * (y-sigmoid(np.dot(w.T,X_train[j])+b)) - ((alpha*w*w)/N)
      grad_dw = gradient_dw(X_train[j], y_train[j], w, b, alpha, N)
      grad_db = y_train[j]-sigmoid(np.dot(w.T,X_train[j])+b)
      w = np.array(w) + (alpha * np.array(grad_dw))
      b = b + (alpha * grad_db)

    # predict the output of x_train[for all data points in X_train] using w and b
    predict_train = []
    for m in range(len(y_train)):
        z = np.dot(w, X_train[m])+b
        predict_train.append(sigmoid(z)) 
    
    # store all the train loss values in a list
    train_loss = logloss(y_train, predict_train)

    # Compare previous loss and current loss, 
    #if loss is not updating then stop the process and return w,b
    if log_loss_train and train_loss > log_loss_train[-1]: 
      return w, b, log_loss_train,
    
    log_loss_train.append(train_loss)

  return w, b, log_loss_train, predict_train

# Coefficient Vector for Alpha=0.01 & 500 Iterations

In [39]:
alpha  = 0.01
epochs = 500
w, b, log_loss_train, predict_train = train(x_train_standard, y_train, epochs, alpha)
print ("Coefficient/Theta/weight vector: ", w)
print ("Intercept: ", b)

Coefficient/Theta/weight vector:  [ 4.35502672 -3.56224611 -0.77656463  0.3253623   4.08582901 -4.90450451
 -2.42579193  0.71340712  0.11008414 -1.85272317  1.40193132 -4.42692   ]
Intercept:  [0.33767034]


# OUTPUT FOR TRAINING DATA

In [40]:
#FOR TRAINING DATA
# predict the output of x_traiin[for all data points in X_train] using w,b
predict_train = []
for m in range(len(y_train)):
    z = np.dot(w, x_train_standard[m])+b
    predict_train.append(sigmoid(z))

predict_train_cl=[]
for i in range(len(predict_train)):
    if(predict_train[i]>=0.5):
        predict_train_cl.append(1)
    if(predict_train[i]<0.5):
        predict_train_cl.append(0)

        
print("PREDICTED OUTPUT FOR TRAINING SET")
print(predict_train_cl,"\n")

#print confusion matrix in form 
#  TP  FP
#  FN  TN
from sklearn.metrics import confusion_matrix
confusion_train=confusion_matrix(y_train, predict_train_cl)
print("'CONFUSION MATRIX' for Training Set is")
print(confusion_train,"\n")


#ACCURACY
totalpredictions=confusion_train[0][0]+confusion_train[0][1]+confusion_train[1][0]+confusion_train[1][1]
accuracy_train=(confusion_train[0][0]+confusion_train[1][1])/totalpredictions
print("Accuracy for Training Set is",accuracy_train,"\n")

#PRECISION
precision_train=confusion_train[0][0]/(confusion_train[0][0]+confusion_train[0][1])
print("Precision for Training Set is",precision_train,"\n")

#RECALL
recall_train=confusion_train[0][0]/(confusion_train[0][0]+confusion_train[1][0])
print("Recall for Training Set is",recall_train)

PREDICTED OUTPUT FOR TRAINING SET
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,

# OUTPUT FOR VALIDATION DATA

In [41]:
#FOR Validation DATA
# predict the output of x_val[for all data points in X_val] using w,b
predict_val = []
for m in range(len(y_val)):
    z = np.dot(w, x_val_standard[m])+b
    predict_val.append(sigmoid(z))

predict_val_cl=[]
for i in range(len(predict_val)):
    if(predict_val[i]>=0.5):
        predict_val_cl.append(1)
    if(predict_val[i]<0.5):
        predict_val_cl.append(0)

        
print("PREDICTED OUTPUT FOR VALIDATION SET")
print(predict_val_cl,"\n")

#print confusion matrix in form 
#  TP  FP
#  FN  TN
from sklearn.metrics import confusion_matrix
confusion_val=confusion_matrix(y_val, predict_val_cl)
print("'CONFUSION MATRIX' for Validation Set is")
print(confusion_val,"\n")

#ACCURACY
totalpredictions=confusion_val[0][0]+confusion_val[0][1]+confusion_val[1][0]+confusion_val[1][1]
accuracy_val=(confusion_val[0][0]+confusion_val[1][1])/totalpredictions
print("Accuracy for Validation Set is",accuracy_val,"\n")

#PRECISION
precision_val=confusion_val[0][0]/(confusion_val[0][0]+confusion_val[0][1])
print("Precision for Validation Set is",precision_val,"\n")

#RECALL
recall_val=confusion_val[0][0]/(confusion_val[0][0]+confusion_val[1][0])
print("Recall for Validation Set is",recall_val,"\n")

PREDICTED OUTPUT FOR VALIDATION SET
[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 

# OUTPUT FOR TESTING DATA

In [42]:
#FOR TESTING DATA
# predict the output of x_traiin[for all data points in X_test] using w,b
predict_test = []
for m in range(len(y_test)):
    z = np.dot(w, x_test_standard[m])+b
    predict_test.append(sigmoid(z))

predict_test_cl=[]
for i in range(len(predict_test)):
    if(predict_test[i]>=0.5):
        predict_test_cl.append(1)
    if(predict_test[i]<0.5):
        predict_test_cl.append(0)

        
print("PREDICTED OUTPUT FOR TESTING SET")
print(predict_test_cl,"\n")

#print confusion matrix in form 
#  TP  FP
#  FN  TN
from sklearn.metrics import confusion_matrix
confusion_test=confusion_matrix(y_test, predict_test_cl)
print("'CONFUSION MATRIX' for Testing Set is")
print(confusion_test,"\n")

#ACCURACY
totalpredictions=confusion_test[0][0]+confusion_test[0][1]+confusion_test[1][0]+confusion_test[1][1]
accuracy_test=(confusion_test[0][0]+confusion_test[1][1])/totalpredictions
print("Accuracy for Testing Set is",accuracy_test,"\n")

#PRECISION
precision_test=confusion_test[0][0]/(confusion_test[0][0]+confusion_test[0][1])
print("Precision for Testing Set is",precision_test,"\n")

#RECALL
recall_test=confusion_test[0][0]/(confusion_test[0][0]+confusion_test[1][0])
print("Recall for Testing Set is",recall_test,"\n")

PREDICTED OUTPUT FOR TESTING SET
[0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 